In [1]:
import numpy as np
import pandas as pd
import simpy
import matplotlib.pyplot as plt

simpy.__version__

'4.0.1'

In [2]:
from sim_utility.distributions import (Exponential, Bernoulli, Triangular, Uniform, Discrete)

In [3]:
seeds = [199769, 778141, 120488, 523288, 447677, 586533, 309253, 332099,
         134200, 218660, 137802, 431191, 709036, 113578, 901009, 662765,
         877876, 952621, 514789, 118072]

In [19]:
#this will be from distributions
MEAN_IAT = 18

#probability that arrival is major
P_WALK_IN_MAJOR =  0.240
P_AMB_MAJOR = 0.875

#MAJORS DISPOSAL 

elements = [0, 1, 2]
probs = [0.34, 0.21, 0.45]

dist = Discrete(elements, probs)
dist.sample()

2

In [14]:
class ScenarioArgs(object):
    def __init__(self):
        pass

In [ ]:
class EmergencyDepartment(object):
    
    def __init__(self, env, args):
        self.env = env
        
   

